# SC DOT Data and Zillow Home Sales - How has traffic in South Carolina changed?

One of my least favorite parts of the day is my daily commute. Anecdotally, I feel as if traffic has worsened/increased in South Carolina since I moved here in 2007, and it's worsening at an increasing rate every year.
I stumbled across some data on the SC-DOT GIS page that I thought might help confirm or deny my suspicion.
Here's where the data can be downloaded - https://www.scdot.org/travel/travel-mappinggis.aspx.
The data in question that I'm examining is Annual Average Daily Traffic counts from the years 2009 - 2018.
This data is recorded at the level of StationID and RouteIdentifier with additional information as well, including Latitude and Longitude of the recording Station.
I downloaded the .zip files for each separately and then renamed them and put them into one folder - shp_files.
The data comes in both .shp and .dbf (Xbase) format.
More about these file types here:
TODO

SCDOT has some interactive ArcGIS maps with these data points plotted already - http://scdot.maps.arcgis.com/apps/MinimalGallery/index.html?appid=7420aa1f39d84400a6d7e8cdaacc89cd

However, these plots don't fully convey (to me) the true amount of traffic, as all station points are plotted as little cars with no information about AADT, nor the change in traffic patterns year over year.
Nor do they address any underlying causes of what may be driving potential traffic pattern changes.

Tangentially, SCDOT does provide a wealth of other data for citizens to browse, some of which look quite interesting.
http://scdot.maps.arcgis.com/apps/MinimalGallery/index.html?appid=e8ace63de0e6423394d04c9c091e893b#
I am particularly interested in how the "South Carolina Roads by Pavement Status" dataset folds into the questions at hand here, but that goes beyond the scope of this post. Perhaps to be addressed later.

Getting back on topic, one of the most intuitive drivers of change in traffic patterns could be popluation growth/decline in the areas nearby. I chose to use a data set from Zillow - https://www.zillow.com/research/data/ - that details Monthly Home Sales by ZipCode as a proxy to population growth per Zip Code.

Finally, I also downloaded a data set from OpenDataSoft - https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/ - that cross references ZipCode to latitude and longitude. I only downloaded data for South Carolina.


### Questions to Answer:

1) What the data looks like and what does it take to get into a unified format?

2) Has overall traffic increased in SC over the past 10 years?

3) Are there are any areas in SC that show more aggressive traffic growth?

4) Does number of home sales in Zip Codes within a certain radius of a station (e.g. 50 km) have any bearing on traffic numbers the next or same year?

In [1]:
# import required libraries

import pandas as pd
import numpy as np
from simpledbf import Dbf5
import os

PyTables is not installed. No support for HDF output.


In [5]:
# change into the directory where the shp files live
os.chdir("./shp_files")


In [6]:
# read GIS dbf data into dataframes, one file for each year between 2009 and 2018
shp_dfs = {}
for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.endswith(".dbf"):
            # print(file.split('.')[0])
            dbf = Dbf5(os.path.join(root, file))
            df = dbf.to_dataframe()
            shp_dfs[file.split('.')[0]] = df

In [7]:
# give an ol look see at the most recent year
shp_dfs['2018'].head()

,CountyNumb,RouteTypeN,RouteType1,RouteNumbe,MeterMileP,BeginMileP,EndMilePoi,StationNum,Termini,FactoredAA,FactoredA1,MapLRS,Status1,ID1,RouteAuxil,CountyName,Long,Lat
0,1.0,2.0,US,178.0,1.196,0.00,3.82,101.0,County Line - ANDERSON TO S- 166 (DRAKE RD),4300.0,2018.0,01020017800E,NaN,1,NaN,ABBEVILLE,-82.38521,34.41979
1,1.0,2.0,US,178.0,4.345,3.82,4.91,103.0,S- 166 (DRAKE RD) TO SC 184 (MAIN ST W),4600.0,2018.0,01020017800E,NaN,2,NaN,ABBEVILLE,-82.35324,34.38344
2,1.0,2.0,US,178.0,5.633,4.91,7.28,105.0,SC 184 (MAIN ST W) TO County Line - GREENWOOD,3600.0,2018.0,01020017800E,NaN,3,NaN,ABBEVILLE,-82.33765,34.37099
3,1.0,4.0,SC,20.0,0.074,0.00,0.18,109.0,"SC 203 (WASHINGTON ST), L- 20, L- 980 TO SC 71...",4900.0,2018.0,01040002000E,NaN,4,NaN,ABBEVILLE,-82.38017,34.17889
4,1.0,4.0,SC,20.0,0.439,0.18,0.45,111.0,SC 71 (N MAIN ST) TO L- 170 (RICHEY ST),2200.0,2018.0,01040002000E,NaN,5,NaN,ABBEVILLE,-82.38114,34.18369


Just by eyeballing, it can be surmised what each field name means, along with the helpful data dictionary supplied by SCDOT.
For example "FactorerA1" is clearly the year of this particular dataset, while "Long" and "Lat" hold the longitude and latitude of the datapoint.

Next, we'll check to see if the column names match across all the dataframes we have, one for each year.

In [8]:
# check if the columns in the dfs match - per df, convert columns to sets. Check the intersection of all sets.
col_sets = map(lambda x: set(x.columns), shp_dfs.values())

In [9]:
# unpack the list of column sets into set.intersection, which returns common elements in set
common_cols = set.intersection(*col_sets)

In [10]:
# check to see what's common between the dfs
common_cols

{'ID1'}

Uh oh! There's only one column that is common between all of the dataframes. Let's dig in a little more.

In [11]:
# Let's check to see what the actual columns are named and how many there are
for df in shp_dfs.values():
    print(df.columns, len(df.columns))

Index(['CountyNumb', 'RouteTypeN', 'RouteType1', 'RouteNumbe', 'MeterMileP',
       'BeginMileP', 'EndMilePoi', 'StationNum', 'Termini', 'FactoredAA',
       'FactoredA1', 'MapLRS', 'Status1', 'ID1', 'RouteAuxil', 'CountyName',
       'Long', 'Lat'],
      dtype='object') 18
Index(['Station_Nu', 'Route_LRS', 'County_ID', 'Route_Type', 'Route_Numb',
       'Route_Auxi', 'Descriptio', 'Count', 'Year', 'ID1'],
      dtype='object') 10
Index(['STATION_NU', 'MILE_POINT', 'ROUTE_LRS', 'MAP_TYPE', 'LATITUDE',
       'LONGITUDE', 'COUNTY_ID', 'ROUTE_TYPE', 'ROUTE_NUMB', 'ROUTE_AUX',
       'COUNT', 'YEAR', 'DESCRIPTIO', 'ID1', 'GMRotation'],
      dtype='object') 15
Index(['CountyName', 'RouteTypeN', 'RouteNumbe', 'RouteAuxil', 'MeterMileP',
       'BegiNMileP', 'EndMilePoi', 'StationNum', 'Termini', 'FactoredAA',
       'FactoredA1', 'MapLRS', 'Status1', 'ID1', 'Latitude', 'Longitude'],
      dtype='object') 16
Index(['STATION', 'MILE_POINT', 'ROUTE_LRS', 'MAP_TYPE', 'ID1', 'LATITUDE',
      

We can immediately see that we have different numbers of columns per year, and that most of the columns are all named differently. We'll try to address that.

In [12]:
# maybe we'll get better results if we do some simple string formatting first
for df in shp_dfs.values():
    df.columns = [c.replace('_', '').lower().strip() for c in df.columns]

In [13]:
# check set intersection again
col_sets = list(map(lambda x: set(x.columns), shp_dfs.values()))
set.intersection(*col_sets)

{'id1'}

Still only one column that's the same! Time to do some brute-force mapping.

In [15]:
for year, df in shp_dfs.items():
    print(year, df.columns, len(df.columns))

2018 Index(['countynumb', 'routetypen', 'routetype1', 'routenumbe', 'metermilep',
       'beginmilep', 'endmilepoi', 'stationnum', 'termini', 'factoredaa',
       'factoreda1', 'maplrs', 'status1', 'id1', 'routeauxil', 'countyname',
       'long', 'lat'],
      dtype='object') 18
2013 Index(['stationnu', 'routelrs', 'countyid', 'routetype', 'routenumb',
       'routeauxi', 'descriptio', 'count', 'year', 'id1'],
      dtype='object') 10
2010 Index(['stationnu', 'milepoint', 'routelrs', 'maptype', 'latitude',
       'longitude', 'countyid', 'routetype', 'routenumb', 'routeaux', 'count',
       'year', 'descriptio', 'id1', 'gmrotation'],
      dtype='object') 15
2016 Index(['countyname', 'routetypen', 'routenumbe', 'routeauxil', 'metermilep',
       'beginmilep', 'endmilepoi', 'stationnum', 'termini', 'factoredaa',
       'factoreda1', 'maplrs', 'status1', 'id1', 'latitude', 'longitude'],
      dtype='object') 16
2015 Index(['station', 'milepoint', 'routelrs', 'maptype', 'id1', 'latitude'

In [19]:
# create a dictionary that will allow us to rename columns from key to value.
# we won't map every column - only keep a subset

col_mapping_dict = {
    **dict.fromkeys(['station', 'stationnu', 'stationnum'], 'station_id'),
    **dict.fromkeys(['latitude', 'lat'], 'latitude'),
    **dict.fromkeys(['longitude', 'long'], 'longitude'), 
    **dict.fromkeys(['aadtyr', 'year', 'factored1', 'factoreda1'], 'year'),
    **dict.fromkeys(['routelrs', 'maplrs'], 'route_identifier'),
    **dict.fromkeys(['termini', 'descriptio'], 'route_leg_descrip'),
    # **dict.fromkeys(['beginmilep', 'beginmile'], 'route_leg_beginmile'),
    # **dict.fromkeys(['endmilepo', 'endmilepoi'], 'route_leg_endmile'),
    **dict.fromkeys(['routetype', 'rtetype', 'routetypen'], 'route_type_id'),   # has to be a numeric column as well, some collision here
    **dict.fromkeys(['rtenum', 'rtenumb', 'routenumb', 'routenum', 'routenumbe'], 'route_number'),
    **dict.fromkeys(['county', 'countyname', 'countynam'], 'county_name'),
    # **dict.fromkeys(['countyid', 'countynumb'], 'county_id'),
    **dict.fromkeys(['aadt', 'factoreda', 'count', 'factoredaa'], 'average_daily_traffic'),
    **dict.fromkeys(['id1'], 'row_number')
}

In [20]:
# rename columns as per mapping dict
shp_dfs_renamed = {year: df.rename(columns=col_mapping_dict) for year, df in shp_dfs.items()}
# drop columns not mapped
shp_dfs_renamed = {year: df.drop([c for c in df.columns if c not in col_mapping_dict.values()], axis=1) for year, df in shp_dfs_renamed.items()}
# drop any duplicated columns
shp_dfs_renamed = {year: df.loc[:, ~df.columns.duplicated()] for year, df in shp_dfs_renamed.items()}

In [21]:
# still some collision - column name can mean different things in different years.
# rename some columns that aren't right
shp_dfs_renamed['2010'].rename(columns={'route_type_id':'route_type'}, inplace=True)
shp_dfs_renamed['2016'].rename(columns={'route_type_id':'route_type'}, inplace=True)
shp_dfs_renamed['2015'].rename(columns={'route_type_id':'route_type'}, inplace=True)
shp_dfs_renamed['2017'].rename(columns={'route_type_id':'route_type'}, inplace=True)
shp_dfs_renamed['2011'].rename(columns={'route_type_id':'route_type'}, inplace=True)
shp_dfs_renamed['2014'].rename(columns={'route_type_id':'route_type'}, inplace=True)
shp_dfs_renamed['2017'].rename(columns={'county_name':'county_id'}, inplace=True)
shp_dfs_renamed['2009'].drop('county_name', axis=1, inplace=True)
shp_dfs_renamed['2012'].drop('county_name', axis=1, inplace=True)

OK - columns should now all be corrently named across years. We also dropped a subset of the columns, only keeping a subset.
Let's check now to make sure that the records we have are unique where they're supposed to be - namely, we should only have one row per year for a stationid/routeid/routenumber combination.

In [24]:
# records should be unique across station_id, route_identifier, and route_number - do some verification
for year, df in shp_dfs_renamed.items():
    temp_df = df.groupby(['station_id', 'route_identifier', 'route_number']).size()
    # print rows where the size of the group is > 1 (duplicates)
    print(year,'\n', temp_df.loc[temp_df > 1.0])

2018 
 station_id  route_identifier  route_number
101.0       32020000100N      1.0             2
133.0       28020000100N      1.0             2
173.0       10020007800E      78.0            2
192.0       30040006600E      66.0            3
            36040006600E      66.0            2
278.0       30090028900N      289.0           2
279.0       15070003600E      36.0            2
306.0       24070005000E      50.0            2
321.0       21070013200E      132.0           2
411.0       14070010400E      104.0           2
413.0       02070004600E      46.0            2
415.0       13090015900N      159.0           2
426.0       10070004600E      46.0            2
429.0       10070010300N      103.0           2
463.0       39070030400E      304.0           2
492.0       16070016400E      164.0           2
495.0       02070018200E      182.0           2
498.0       04070009800E      98.0            2
509.0       02070030900N      309.0           2
551.0       42090056300N      563.0   

It would appear that we have some duplicates, or we think we have duplicates. We'll define a quick function to spot check some records and get a sense of why rows are duplicated.

In [26]:
# we can spot check some of these to see what's up with the records
def check_records(year, station_id, route_identifier):
    mask = (shp_dfs_renamed[year]['station_id'] == station_id) & (shp_dfs_renamed[year]['route_identifier'] == route_identifier)
    return shp_dfs_renamed[year].loc[mask]

check_records('2011', 2520.0, '08010052600E')

,station_id,average_daily_traffic,route_leg_descrip,year,route_type,route_number,route_identifier,latitude,longitude,row_number
1675,2520.0,53400.0,SEVEN FARMS RD TO S- 97 (CHARLESTON),2011.0,I-,526.0,08010052600E,32:51:39.5474,-79:53:52.4178,1676
13028,2520.0,53400.0,SEVEN FARMS RD TO S- 97 (CHARLESTON),2011.0,I-,526.0,08010052600E,32:51:39.5474,-79:53:52.4178,1676


Eyeballing a few sets of records leads me to belive that we are in fact dealing with duplicate rows. Most are only unique based on the row number. We'll drop our duplicates to make sure they don't sully the dataset.

In [27]:
# eyeballing the records reveals that they are duplicated rows. we'll take the first from every group
# head(1) will return the first row per group
for year, df in shp_dfs_renamed.items():
    temp_df = df.groupby(['station_id', 'route_identifier', 'route_number']).head(1)
    shp_dfs_renamed[year] = temp_df

Check for duplicates again.

In [29]:
#check for dupes again
for year, df in shp_dfs_renamed.items():
    temp_df = df.groupby(['station_id', 'route_identifier', 'route_number']).size()
    print(year,'\n', temp_df.loc[temp_df > 1.0])

2018 
 Series([], dtype: int64)
2013 
 Series([], dtype: int64)
2010 
 Series([], dtype: int64)
2016 
 Series([], dtype: int64)
2015 
 Series([], dtype: int64)
2009 
 Series([], dtype: int64)
2017 
 Series([], dtype: int64)
2012 
 Series([], dtype: int64)
2011 
 Series([], dtype: int64)
2014 
 Series([], dtype: int64)


In [30]:
# verify we took the first row in the duplicated group, and that we only have one row returning.
check_records('2011', 2520.0, '08010052600E')

,station_id,average_daily_traffic,route_leg_descrip,year,route_type,route_number,route_identifier,latitude,longitude,row_number
1675,2520.0,53400.0,SEVEN FARMS RD TO S- 97 (CHARLESTON),2011.0,I-,526.0,08010052600E,32:51:39.5474,-79:53:52.4178,1676


At this point we've realized that the data can very quite widely from year to year. In order to clean the data up a little bit more, we will try to standardize the data across years. To accomplish this we'll use pandas' update method - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.update.html - and overwrite the values in every year with our standardized values. Because the 2017 and 2018 dataframes don't have all the columns that we're interested in looking at, we'll use the 2016 dataframe to update/overwrite values in a specific subset of columns across all years.

In [31]:
# set index of all dfs to the unique identifiers
for year, df in shp_dfs_renamed.items():
    temp_df = df.set_index(['station_id', 'route_identifier', 'route_number'])
    shp_dfs_renamed[year] = temp_df

In [32]:
# fill missing year value in any dfs
for year, df in shp_dfs_renamed.items():
    df['year'] = df.year.fillna(year)

In [33]:
# 2018 and 2017 dfs don't have all the columns, so use the 2016 df to standardize fields
# check how many nas in each columns in the 2016 df
shp_dfs_renamed['2016'].isna().sum()

county_name                0
route_type                 0
route_leg_descrip          0
average_daily_traffic      0
year                       0
row_number                 0
latitude                 179
longitude                179
dtype: int64

In [34]:
# we have some NaNs for lat/long in the 2016 df, so we'll try and reduce those as much as we can.
# update 2016 latitudes/longitudes with 2018 latitudes/longitudes where stationid, route_id, and route_number match (index of each df)

shp_dfs_renamed['2016'].update(shp_dfs_renamed['2018'][['latitude', 'longitude']])

# still some nulls - try the 2015 df
shp_dfs_renamed['2016'].update(shp_dfs_renamed['2015'][['latitude', 'longitude']])

In [35]:
# check nas again
shp_dfs_renamed['2016'].isna().sum()

county_name                0
route_type                 0
route_leg_descrip          0
average_daily_traffic      0
year                       0
row_number                 0
latitude                 153
longitude                153
dtype: int64

In [36]:
# 2018 and 2017 dfs don't have all the columns, so use the 2016 df to standardize fields
# we're standardizing data across the five following columns - DON'T want to overwrite AADT.
cols_to_update = ['route_type', 'route_leg_descrip', 'latitude', 'longitude', 'county_name']
update_df = shp_dfs_renamed['2016'][cols_to_update]

for year, df in shp_dfs_renamed.items():
    df.update(update_df, overwrite=True)

At this point we should be ready to combine all the dataframes together.

In [37]:
# stack all the data frames together
traffic_df = pd.concat(shp_dfs_renamed.values(), sort=True, axis=0).reset_index()

In [38]:
# eyeball the data 
traffic_df.head()

,station_id,route_identifier,route_number,average_daily_traffic,county_id,county_name,latitude,longitude,route_leg_descrip,route_type,route_type_id,row_number,year
0,101.0,01020017800E,178.0,4300.0,NaN,ABBEVILLE,34.41979,-82.38521,County Line - ANDERSON TO S- 166 (DRAKE RD),NaN,2.0,1,2018
1,103.0,01020017800E,178.0,4600.0,NaN,ABBEVILLE,34.38344,-82.35325,S- 166 (DRAKE RD) TO SC 184 (N MAIN ST),NaN,2.0,2,2018
2,105.0,01020017800E,178.0,3600.0,NaN,ABBEVILLE,34.37099,-82.33765,SC 184 (N MAIN ST) TO County Line - GREENWOOD,NaN,2.0,3,2018
3,109.0,01040002000E,20.0,4900.0,NaN,ABBEVILLE,34.17888,-82.38016,"SC 203 (WASHINGTON ST), L- 20, L- 980 TO SC 71",NaN,4.0,4,2018
4,111.0,01040002000E,20.0,2200.0,NaN,ABBEVILLE,34.18359,-82.38115,SC 71 TO L- 170,NaN,4.0,5,2018


In [39]:
# drop columns I actually don't want
traffic_df = traffic_df.drop(['county_id', 'route_type_id', 'row_number'], axis=1)

In [40]:
traffic_df.head()

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year
0,101.0,01020017800E,178.0,4300.0,ABBEVILLE,34.41979,-82.38521,County Line - ANDERSON TO S- 166 (DRAKE RD),NaN,2018
1,103.0,01020017800E,178.0,4600.0,ABBEVILLE,34.38344,-82.35325,S- 166 (DRAKE RD) TO SC 184 (N MAIN ST),NaN,2018
2,105.0,01020017800E,178.0,3600.0,ABBEVILLE,34.37099,-82.33765,SC 184 (N MAIN ST) TO County Line - GREENWOOD,NaN,2018
3,109.0,01040002000E,20.0,4900.0,ABBEVILLE,34.17888,-82.38016,"SC 203 (WASHINGTON ST), L- 20, L- 980 TO SC 71",NaN,2018
4,111.0,01040002000E,20.0,2200.0,ABBEVILLE,34.18359,-82.38115,SC 71 TO L- 170,NaN,2018


Eyeballing the head indicates that there are STILL NaNs in the data - the index of our update df (stationid, routeid, routenumber) did not overlap every possible combination in the data across the years. We can handle these NaNs with a similar operation - grouping the data by our unique identifers and then filling within groups

In [41]:
# now fill in cols that are still na by unique id - NOT the average traffic column (main data we care about)
# fill nas by group
cols_to_fill = ['county_name', 'latitude', 'longitude', 'route_leg_descrip', 'route_type']
for col in cols_to_fill:
    traffic_df[col] = traffic_df.groupby(['station_id', 'route_identifier', 'route_number'])[col].ffill().bfill()

In [42]:
# check for any nas remaining
traffic_df.isna().sum()

station_id                0
route_identifier          0
route_number              0
average_daily_traffic    69
county_name               0
latitude                  0
longitude                 0
route_leg_descrip         0
route_type                0
year                      0
dtype: int64

In [43]:
# drop remaining nas - no data!
traffic_df = traffic_df.dropna()

In [ ]:
# do some plotting on pct changes etc

In [ ]:
# can't plot all lats and longs because the lats/longs are strings.

In [46]:
for year, df in shp_dfs_renamed.items():
    try:
        print(year, df.latitude.str.contains(':').sum())
    except:
        pass
update_df.latitude.str.contains(':').sum()

2018 0
2010 404
2016 0
2015 0
2009 442
2012 359
2011 371
2014 163


0

In [47]:
traffic_df.loc[traffic_df.latitude.str.contains(':')]

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year,traffic_yearly_pct_change
103549,100.0,08020001702S,17.0,35500.0,Berkeley,33:2:13.817,-80:9:13.399,County Line - DORCHESTER TO I- 26,US,2014,NaN
58253,106.0,07020001706N,17.0,4700.0,BEAUFORT,32:36:03.3216,-80:45:31.8595,U.S. 17 TO U.S. 21,US,2009,NaN
24188,106.0,07020001706N,17.0,4100.0,BEAUFORT,32:36:03.3216,-80:45:31.8595,U.S. 17 TO U.S. 21,US,2010,-0.127660
92058,106.0,07020001706N,17.0,3700.0,BEAUFORT,32:36:03.3216,-80:45:31.8595,US 17 TO US 21,US,2011,-0.097561
80804,106.0,07020001706N,17.0,3700.0,BEAUFORT,32:36:03.3216,-80:45:31.8595,US 17 TO US 21,US,2012,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
92990,2521.0,10010052600W,526.0,22200.0,CHEROKEE,32:49:14.6714,-79:50:55.6515,S- 56 TO US 17,I-,2011,0.067308
81738,2521.0,10010052600W,526.0,21800.0,CHEROKEE,32:49:14.6714,-79:50:55.6515,S- 56 TO US 17,I-,2012,-0.018018
104258,2521.0,10010052600W,526.0,25000.0,Charleston,32:49:14.671,-79:50:55.652,S- 56 TO US 17,I-,2014,0.050420
59176,2523.0,10010052600E,526.0,14900.0,CHEROKEE,32:48:47.4084,-79:50:58.4613,US 17 To End,I-,2009,NaN


In [50]:
def convert_lat_long(string):
    split = string.split(':')
   
    converted = float(split[0]) + float(split[1])/60 + float(split[2])/(60*60)
    
    return converted

test = '33:2:13.817'

convert_lat_long(test)


33.037171388888886

In [51]:
traffic_df.loc[traffic_df.latitude.str.contains(':'), 'latitude'] = traffic_df.loc[traffic_df.latitude.str.contains(':'), 'latitude'].apply(convert_lat_long) 

In [52]:
traffic_df.loc[traffic_df.longitude.str.contains(':'), 'longitude'] = traffic_df.loc[traffic_df.longitude.str.contains(':'), 'longitude'].apply(convert_lat_long) 

In [53]:
traffic_df['latitude'] = traffic_df.latitude.astype('float')

In [54]:
traffic_df['longitude'] = traffic_df.longitude.astype('float')

In [44]:
# get the pct change year over year for average daily traffic
traffic_df['traffic_yearly_pct_change'] = traffic_df \
    .sort_values(['station_id', 'route_identifier', 'route_number', 'year']) \
    .groupby(['station_id', 'route_identifier', 'route_number']) \
    .average_daily_traffic \
    .pct_change()

In [45]:
traffic_df = traffic_df.sort_values(['station_id', 'route_identifier', 'route_number', 'year'])

In [36]:
traffic_df[traffic_df['traffic_yearly_pct_change'] > 0.2]

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year,traffic_yearly_pct_change
46302,100.0,04020002900N,29.0,4000.0,ANDERSON,34.35590,-82.81412,State Line - GEORGIA TO SC 187 (HIGHWAY 187 S),US,2015,0.290323
69660,100.0,08020001702N,17.0,53100.0,BERKELEY,33.03701,-80.15366,County Line - DORCHESTER TO I- 26,US,2017,0.288835
107867,100.0,28040001200E,12.0,4100.0,KERSHAW,34.11999,-80.77968,County Line - RICHLAND TO S- 47 (FORT JACKSON RD),SC,2014,0.413793
46303,101.0,04020002900N,29.0,2100.0,ANDERSON,34.40918,-82.79075,SC 187 (HIGHWAY 187 S) TO US 29 BUS (HIGHWAY ...,US,2015,0.312500
35410,101.0,07020001700N,17.0,12400.0,BEAUFORT,32.64004,-80.85637,County Line - JASPER TO US 17 ALT (CASTLE HALL...,US,2016,0.441860
...,...,...,...,...,...,...,...,...,...,...,...
50675,2435.0,23010018500N,185.0,5000.0,GREENVILLE,34.77331,-82.44549,SC 153 (153 HWY) TO I- 85,I-,2015,0.250000
95757,2439.0,23010018500N,185.0,16700.0,GREENVILLE,34.80352,-82.42446,"US 25 (WHITE HORSE RD) TO , SC 20",I-,2011,0.590476
9271,2489.0,42010058500S,585.0,32200.0,SPARTANBURG,34.97494,-81.94188,"US 176 CO2 (N CHURCH ST), SC 9 TO US 221 (WHIT...",SC,2018,0.201493
9272,2491.0,42010058500S,585.0,31200.0,SPARTANBURG,34.97158,-81.93761,US 221 (WHITNEY RD) TO US 176 (N PINE ST),SC,2018,0.214008


### Zillow Sales data by Zip Code

In [59]:
# navigate up
os.chdir('..')

In [68]:
# read in the data
sales_df = pd.read_csv('Sale_Counts_Zip.csv')
sales_df.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,seasAdj
0,61639,10025,New York,1,NaN,NaN,NaN,NaN,NaN,NaN,...,76.0,33.0,47.0,56.0,35.0,70.0,78.0,66.0,63.0,0
1,84654,60657,Illinois,2,NaN,NaN,NaN,NaN,NaN,NaN,...,91.0,77.0,113.0,157.0,189.0,165.0,186.0,141.0,152.0,0
2,61637,10023,New York,3,NaN,NaN,NaN,NaN,NaN,NaN,...,80.0,45.0,63.0,45.0,66.0,85.0,79.0,90.0,95.0,0
3,91982,77494,Texas,4,56.0,71.0,84.0,95.0,116.0,86.0,...,86.0,112.0,186.0,218.0,200.0,204.0,245.0,226.0,NaN,0
4,84616,60614,Illinois,5,NaN,NaN,NaN,NaN,NaN,NaN,...,75.0,85.0,144.0,163.0,219.0,209.0,204.0,196.0,173.0,0


The data is very wide - sales are recording in columns. Our steps for prepping this data will be to:
1) Filter down to only South Carolina Zip Codes

2) Drop columns we don't care about

3) Convert records from wide to tall format

4) Get the sum of sales per zipcode per year

In [69]:
# filter to only SC
sales_df = sales_df.loc[sales_df.StateName == 'South Carolina']

# drop columns we don't care about and rename columns
sales_df = sales_df.drop(['RegionID', 'StateName', 'SizeRank', 'seasAdj'], axis=1)
sales_df.rename(columns={'RegionName': 'ZipCode'}, inplace=True)
sales_df = sales_df.set_index('ZipCode')

In [70]:
sales_df.head()

,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,...,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09
ZipCode,,,,,,,,,,,,,,,,,,,,,
29732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.0,75.0,75.0,134.0,140.0,125.0,136.0,128.0,138.0,151.0
29072,63.0,89.0,75.0,70.0,76.0,65.0,68.0,47.0,42.0,43.0,...,165.0,88.0,88.0,126.0,132.0,192.0,156.0,180.0,169.0,142.0
29730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,119.0,71.0,60.0,91.0,95.0,116.0,111.0,87.0,100.0,107.0
29464,58.0,78.0,73.0,88.0,81.0,73.0,57.0,69.0,49.0,35.0,...,76.0,63.0,91.0,94.0,138.0,116.0,135.0,146.0,141.0,NaN
29681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,99.0,118.0,150.0,129.0,165.0,136.0,151.0,150.0,161.0


In [71]:
# stack all sales into one column
sales_df = sales_df.stack().reset_index()
sales_df.columns = ['ZipCode', 'YearMonth', 'Sales']
sales_df

,ZipCode,YearMonth,Sales
0,29732,2014-08,127.0
1,29732,2014-09,145.0
2,29732,2014-10,124.0
3,29732,2014-11,121.0
4,29732,2014-12,124.0
...,...,...,...
51258,820,2019-05,0.0
51259,820,2019-06,0.0
51260,820,2019-07,0.0
51261,820,2019-08,0.0


In [76]:
# extract the year out of the Sales record
sales_df['Year'] = sales_df['YearMonth'].str.split('-').str[0]
sales_df.head()

,ZipCode,YearMonth,Sales,Year
0,29732,2014-08,127.0,2014
1,29732,2014-09,145.0,2014
2,29732,2014-10,124.0,2014
3,29732,2014-11,121.0,2014
4,29732,2014-12,124.0,2014


In [79]:
# yearly sales calculated - sum over ZipCode and year
yearly_sales = sales_df.groupby(['ZipCode', 'Year']).Sales.sum()
yearly_sales.tail(15)

ZipCode  Year
29944    2017    42.0
         2018    28.0
         2019     3.0
29945    2008    22.0
         2009    20.0
         2010    20.0
         2011    25.0
         2012    30.0
         2013    23.0
         2014    23.0
         2015    26.0
         2016    14.0
         2017    23.0
         2018    17.0
         2019     3.0
Name: Sales, dtype: float64

### Zip to Lat/Long xref

In [82]:
# read in the data
zip_xref = pd.read_csv('sc-zip-code-latitude-and-longitude.csv')

In [83]:
zip_xref.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,Unnamed: 8
0,29607,Greenville,SC,34.825592,-82.34099,-5,1,34.825592,-82.34099
1,29164,Wagener,SC,33.659078,-81.40845,-5,1,33.659078,-81.40845
2,29325,Clinton,SC,34.470115,-81.86761,-5,1,34.470115,-81.86761
3,29520,Cheraw,SC,34.688620,-79.92315,-5,1,34.688620,-79.92315
4,29615,Greenville,SC,34.866801,-82.31739,-5,1,34.866801,-82.31739


In [84]:
# merge the zip code to lat/long xref to the yearly sales data
merged_sales = yearly_sales.reset_index().merge(zip_xref, how='left', left_on='ZipCode', right_on='Zip')

In [85]:
# drop zip codes that have a length less than 5
merged_sales = merged_sales.loc[merged_sales.ZipCode.apply(lambda x: len(str(x)) >= 5)]

In [86]:
# drop columns i don't care about
merged_sales = merged_sales.drop(['Zip', 'Timezone', 'Daylight savings time flag', 'geopoint', 'Unnamed: 8'], axis=1)

In [87]:
merged_sales.head(25)

,ZipCode,Year,Sales,City,State,Latitude,Longitude
12,29001,2008,10.0,Alcolu,SC,33.769930,-80.17278
13,29001,2009,10.0,Alcolu,SC,33.769930,-80.17278
14,29001,2010,11.0,Alcolu,SC,33.769930,-80.17278
15,29001,2011,8.0,Alcolu,SC,33.769930,-80.17278
16,29001,2012,9.0,Alcolu,SC,33.769930,-80.17278
17,29001,2013,12.0,Alcolu,SC,33.769930,-80.17278
18,29001,2014,10.0,Alcolu,SC,33.769930,-80.17278
19,29001,2015,3.0,Alcolu,SC,33.769930,-80.17278
20,29001,2016,14.0,Alcolu,SC,33.769930,-80.17278
21,29001,2017,12.0,Alcolu,SC,33.769930,-80.17278


At this point I'd like to calculate how far each Zip Code is from each station. 
I searched online for a way to calculate the as-the-crow-flies distance between two lat/long points.
The first reference I found referred to the "haversine" formula, detailed here - https://www.movable-type.co.uk/scripts/latlong.html. 
I found a nice numpy implmentation on Stack Overflow here: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas from derricw.

In order to calculate the distance between each possible combination of points, I will be taking all unique lat/long points from traffic data and crossjoining with all unique lat/long points from zip code data. Then, I will apply the haversine formula to get the linear distance between each pair of points.

In [89]:
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [91]:
# get unique zip to lat/long
zip_xref['Latitude'] = zip_xref.Latitude.astype('float')
zip_xref['Longitude'] = zip_xref.Longitude.astype('float')
unique_zip = zip_xref.loc[:, ['Zip', 'Latitude', 'Longitude']].drop_duplicates()

In [92]:
unique_zip

,Zip,Latitude,Longitude
0,29607,34.825592,-82.34099
1,29164,33.659078,-81.40845
2,29325,34.470115,-81.86761
3,29520,34.688620,-79.92315
4,29615,34.866801,-82.31739
...,...,...,...
549,29592,34.283207,-79.47272
550,29646,34.169781,-82.15474
551,29142,33.462378,-80.50903
552,29449,32.715745,-80.26738


In [93]:
# get the unique station/routeid/route number lat/long
unique_station = traffic_df \
    .loc[:, ['station_id', 'route_identifier', 'route_number', 'latitude', 'longitude']] \
    .drop_duplicates()

In [94]:
# create the cartesian product/crossjoin by merging on a uniform key
unique_zip['key'] = 0
unique_station['key'] = 0

station_zip_xref = unique_station.merge(unique_zip, on='key').drop('key', axis=1)
station_zip_xref.columns = [
        'station_id',
        'route_identifier',
        'route_number',
        'station_lat',
        'station_long',
        'Zip',
        'zip_lat',
        'zip_long'
    ]

station_zip_xref

,station_id,route_identifier,route_number,station_lat,station_long,Zip,zip_lat,zip_long
0,100.0,01040018500N,185.0,34.37925,-82.44926,29607,34.825592,-82.34099
1,100.0,01040018500N,185.0,34.37925,-82.44926,29164,33.659078,-81.40845
2,100.0,01040018500N,185.0,34.37925,-82.44926,29325,34.470115,-81.86761
3,100.0,01040018500N,185.0,34.37925,-82.44926,29520,34.688620,-79.92315
4,100.0,01040018500N,185.0,34.37925,-82.44926,29615,34.866801,-82.31739
...,...,...,...,...,...,...,...,...
7061833,2607.0,02010052000E,520.0,33.56019,-81.92848,29592,34.283207,-79.47272
7061834,2607.0,02010052000E,520.0,33.56019,-81.92848,29646,34.169781,-82.15474
7061835,2607.0,02010052000E,520.0,33.56019,-81.92848,29142,33.462378,-80.50903
7061836,2607.0,02010052000E,520.0,33.56019,-81.92848,29449,32.715745,-80.26738


In [95]:
station_zip_xref['distance_km'] = haversine_np(
    station_zip_xref.station_long.values,
    station_zip_xref.station_lat.values,
    station_zip_xref.zip_long.values,
    station_zip_xref.zip_lat.values
)

In [96]:
station_zip_xref

,station_id,route_identifier,route_number,station_lat,station_long,Zip,zip_lat,zip_long,distance_km
0,100.0,01040018500N,185.0,34.37925,-82.44926,29607,34.825592,-82.34099,50.578796
1,100.0,01040018500N,185.0,34.37925,-82.44926,29164,33.659078,-81.40845,124.877525
2,100.0,01040018500N,185.0,34.37925,-82.44926,29325,34.470115,-81.86761,54.263882
3,100.0,01040018500N,185.0,34.37925,-82.44926,29520,34.688620,-79.92315,233.784632
4,100.0,01040018500N,185.0,34.37925,-82.44926,29615,34.866801,-82.31739,55.504933
...,...,...,...,...,...,...,...,...,...
7061833,2607.0,02010052000E,520.0,33.56019,-81.92848,29592,34.283207,-79.47272,240.272498
7061834,2607.0,02010052000E,520.0,33.56019,-81.92848,29646,34.169781,-82.15474,70.885076
7061835,2607.0,02010052000E,520.0,33.56019,-81.92848,29142,33.462378,-80.50903,131.964450
7061836,2607.0,02010052000E,520.0,33.56019,-81.92848,29449,32.715745,-80.26738,180.818763


At this point I can calculate the home sales within a certain radius of a station for the current year or previous year and see if that has any bearing on traffic patterns.

In [98]:
def calc_home_sales(traffic_df, sales_df, zip_xref, radius_cutoff, curr_or_prev='curr'):
    filtered_zips = zip_xref.loc[zip_xref.distance_km <= radius_cutoff]
    sales_modified = sales_df.copy(deep=True)
    sales_modified['next_year'] = sales_modified.Year + 1
    sales_per_station = zip_xref.merge(sales_modified, how='left', left_on='Zip', right_on='ZipCode')
    sales_per_station = sales_per_station.groupby(['station_id', 'route_identifier', 'route_number'])
    
    

,ZipCode,Year,Sales,City,State,Latitude,Longitude
12,29001,2008,10.0,Alcolu,SC,33.769930,-80.17278
13,29001,2009,10.0,Alcolu,SC,33.769930,-80.17278
14,29001,2010,11.0,Alcolu,SC,33.769930,-80.17278
15,29001,2011,8.0,Alcolu,SC,33.769930,-80.17278
16,29001,2012,9.0,Alcolu,SC,33.769930,-80.17278
...,...,...,...,...,...,...,...
4507,29945,2015,26.0,Yemassee,SC,32.681058,-80.83348
4508,29945,2016,14.0,Yemassee,SC,32.681058,-80.83348
4509,29945,2017,23.0,Yemassee,SC,32.681058,-80.83348
4510,29945,2018,17.0,Yemassee,SC,32.681058,-80.83348


In [100]:
merged_sales, station_zip_xref

(      ZipCode  Year  Sales      City State   Latitude  Longitude
 12      29001  2008   10.0    Alcolu    SC  33.769930  -80.17278
 13      29001  2009   10.0    Alcolu    SC  33.769930  -80.17278
 14      29001  2010   11.0    Alcolu    SC  33.769930  -80.17278
 15      29001  2011    8.0    Alcolu    SC  33.769930  -80.17278
 16      29001  2012    9.0    Alcolu    SC  33.769930  -80.17278
 ...       ...   ...    ...       ...   ...        ...        ...
 4507    29945  2015   26.0  Yemassee    SC  32.681058  -80.83348
 4508    29945  2016   14.0  Yemassee    SC  32.681058  -80.83348
 4509    29945  2017   23.0  Yemassee    SC  32.681058  -80.83348
 4510    29945  2018   17.0  Yemassee    SC  32.681058  -80.83348
 4511    29945  2019    3.0  Yemassee    SC  32.681058  -80.83348
 
 [4500 rows x 7 columns],
          station_id route_identifier  route_number  station_lat  station_long  \
 0             100.0     01040018500N         185.0     34.37925     -82.44926   
 1             1